This notebook loads MPI LEMON MRI_Preprocessed_Derivatives data.

In [1]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd

In [30]:
DATASET_ROOT_PATH = Path('/mnt/Lifestream/MPI-LEMON/MRI_Preprocessed_Derivatives/')

fmri_files = DATASET_ROOT_PATH.glob('**/func/*MNI2mm.nii.gz')

subject_folders = DATASET_ROOT_PATH.glob('**/sub-*/')

subjects_without_fmri = (
    set(map(lambda x: x.stem, subject_folders)) - set(map(lambda x: x.parents[1].stem, files))
)


PosixPath('/mnt/Lifestream/MPI-LEMON/MRI_Preprocessed_Derivatives/sub-010148/func/sub-010148_ses-01_task-rest_acq-AP_run-01_MNI2mm.nii.gz')

In [73]:
SUBJECT = 'sub-010002'

t1 = DATASET_ROOT_PATH / SUBJECT / 'anat' / f'{SUBJECT}_ses-01_acq-mp2rage_brain.nii.gz'
t2_native = DATASET_ROOT_PATH / SUBJECT / 'func' / f'{SUBJECT}_ses-01_task-rest_acq-AP_run-01_native.nii.gz'
t2_mni = DATASET_ROOT_PATH / SUBJECT / 'func' / f'{SUBJECT}_ses-01_task-rest_acq-AP_run-01_MNI2mm.nii.gz'

confounds = DATASET_ROOT_PATH / SUBJECT / 'func' / f'{SUBJECT}_ses-01_task-rest_acq-AP_run-01_confounds.txt'
confounds = pd.read_csv(confounds, sep=' ', header=None)

from nilearn import image, plotting, maskers

masker = maskers.NiftiMasker(mask_strategy='epi')
t2_native_clean = masker.fit_transform(t2_native)

In [77]:

from src.acnets.parcellations.dosenbach import load_dosenbach2010_masker

atlas_masker, _ = load_dosenbach2010_masker()

# TODO resample native to MNI

ts = atlas_masker.fit_transform(t2_native, confounds)


In [79]:
ts.shape

(652, 160)

In [42]:
# resample to MNI2mm

from nilearn.datasets import load_mni152_template
from nilearn.image import resample_to_img

mni152 = load_mni152_template(resolution=2)
t2_native_resampled = resample_to_img(t2_native, mni152)
t2_native.shape

/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/nilearn/image/resampling.py:663: RuntimeWarning: NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
  _resample_one_img(
/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/nilearn/image/resampling.py:663: RuntimeWarning: NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
  _resample_one_img(
/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/nilearn/image/resampling.py:663: RuntimeWarning: NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
  _resample_one_img(
/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/nilearn/image/resampling.py:663: RuntimeWarning: NaNs or infinite values are present in the data passed

KeyboardInterrupt: 